# Agents for Amazon Bedrock - Testing Agent Invocation

This notebook provides sample code for testing the agent invocation using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html) client for `bedrock-agent-runtime`

### Use Case
Now that we've created the agent and attached the knowledge base to it, we will use the runtime client to invoke the agent with different queries. The boto3 sdk for agent is divided into two clients: `bedrock-agent` and `bedrock-agent-runtime`. The `bedrock-agent` client is responsible for the functionalities to create, update, delete and/or prepare an agent or a knowledge base. While the `bedrock-agent-runtime` is responsible for invoke an agent (with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API) and retrieve documents from a knowledge base (with the [`retrieve`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [`retrieve_and_generate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) APIs). This notebook will focus on the runtime invocation of the agent created in the previous notebooks. We will use the `invoke_agent` API.



### Notebook Walkthrough

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Invoke agent with a new session using Knowledge Bases and Action Groups
- Invoke agent with an existent session
- Invoke agent with prompt attributes
- Invoke agent with trace enabled
- Invoke agent with different languages for the request


### Next Steps: 
In the next lab, we will clean up the resources created

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [41]:
%store -r

#### Importing required packages and initiate variables

Let's also import the necessary packages, setup the logging and initiate the boto3 client for `bedrock-agent-runtime`. We are also initiating the client for `dynamodb` as a support functionality to check that the tasks were performed correctly by our agent

In [42]:
import boto3
import logging
import pprint
import json
import pandas as pd
from agent import invoke_agent_helper
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
dynamodb = boto3.resource('dynamodb')

#### Create support invokeAgent function

Let's again use the same support function called `invoke_agent_helper` from `agents.py` to allow us to invoke the agent with or without trace enabled and with or without session state. 

This function allows the user to send a `query` to the agent with a `session_id`. The user can then decide to enable trace or not using the `enable_trace` boolean variable and to pass a session state as a dictionary via the `session_state` variable.

If a new `session_id` is provided, the agent will create a new conversation without previous context. If the same `session_id` is reused, the conversation history related to that session id is made available to the agent.

If the `enable_trace` is set to `True`, each response from the agent is accompanied by a *trace* that details the step being orchestrated by the agent. It allows you to follow the agent's (reasoning via Chain of Thoughts prompting) that led to the final response at that point of the conversation.

Finally, you can also pass a session context using the `session_state` parameter. The session state allows you to share the following information with the agent:
- **`sessionAttributes`**: attributes that persist over a session between the user and the agent. All invokeAgent calls with the same session_id belong to the same sesison and will have the sessionAttributes shared with them as long as the session time limit has not being surpassed and the user has not ended the session. The sessionAttributes are available in the lambda function but are **not** added to the agent's prompt. As a result, you can only use session attributes if your lambda function can handle them. You can find more examples of using a session attribute [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/06-prompt-and-session-attributes). It is also a good pattern to implement fine-grained access control for certain APIs using the lambda function integration. You can find an example for it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/09-fine-grained-access-permissions)
- **`promptSessionAttributes`**: attributes that persist over a single invokeAgent call. Prompt attributes are added to the prompt and to the lambda function. You can also use the `$prompt_session_attributes$` placeholder when editing the orchestration base prompt.
- **`invocationId`**: The id returned by the agent in the [ReturnControlPayload](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_ReturnControlPayload.html) object in the returnControl field of the InvokeAgent response. This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).
- **`returnControlInvocationResults`**: the results obtained from invoking the action outside of agents for Amazon Bedrock.  This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour

In [43]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [44]:
# test function invocation
items = selectAllFromDynamodb()
items

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,6dca9f6d,Anna


### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

In [45]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What is in the childrens menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The children's menu at The Regrettable Experience includes the following items:

Entrees:
1. Chicken nuggets (allergens: gluten, possible soy)
2. Macaroni and cheese (allergens: dairy, gluten)
3. Mini cheese quesadillas (allergens: dairy, gluten)
4. Peanut butter and banana sandwich (allergens: nuts, gluten)
5. Veggie pita pockets (allergens: gluten, possible soy)

Mains:
1. Mini cheeseburgers (allergens: dairy, gluten)
2. Fish sticks (allergens: gluten, possible soy)
3. Grilled cheese sandwich (allergens: dairy, gluten)
4. Spaghetti with marinara sauce (allergen: gluten)
5. Mini pita pizza (allergens: dairy, gluten)

Desserts:
1. Mini ice cream sundae (allergen: dairy)
2. Fruit kabobs (no allergens)
3. Chocolate chip cookie bites (allergens: dairy, gluten)
4. Banana split (allergen: dairy)
5. Jello cups (no allergens)

CPU times: user 16.4 ms, sys: 702 μs, total: 17.1 ms
Wall time: 12.1 s


### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [46]:
%%time
query = "Which of those options are vegetarian?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Based on the menu items listed, the following options appear to be vegetarian:

Entrees:
- Macaroni and cheese
- Mini cheese quesadillas 
- Peanut butter and banana sandwich
- Veggie pita pockets

Mains: 
- Spaghetti with marinara sauce
- Mini pita pizza

Desserts:
- Fruit kabobs
- Jello cups
CPU times: user 17.4 ms, sys: 392 μs, total: 17.8 ms
Wall time: 3.77 s


As you can see, the agent knows that we are talking about the children's menu.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [47]:
%%time
query = "Hi, I am Maria. I want to create a booking for 4 people, at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Thank you Maria, your reservation for 4 people on May 5th, 2024 at 9pm has been booked successfully. Your booking ID is bd2554b2. Please let me know if you need any other assistance regarding your reservation.
CPU times: user 19.3 ms, sys: 185 μs, total: 19.4 ms
Wall time: 11.5 s


Let's double check that the data was properly added to the dynamoDB table

In [48]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,6dca9f6d,Anna
1,4,2024-05-05,21:00,bd2554b2,Maria


Great! We've used our agent to do create a reservation. However, often when booking tables in restaurants we are already logged in to a system that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [promptSessionAttributes](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [49]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking for 2 people on May 6th, 2024 at 8pm has been created successfully. Your booking ID is effe661b.
CPU times: user 17.8 ms, sys: 2.83 ms, total: 20.6 ms
Wall time: 8.44 s


Again let's validate the the correct data was added to dynamoDB

In [50]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,6dca9f6d,Anna
1,4,2024-05-05,21:00,bd2554b2,Maria
2,2,2024-05-06,20:00,effe661b,John


We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [51]:
%%time
query = "Get the details for the last booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Here are the details for your booking with ID effe661b:
- Date: May 6th, 2024
- Time: 8:00 PM 
- Number of Guests: 2
- Name on Reservation: John
CPU times: user 23.2 ms, sys: 445 μs, total: 23.6 ms
Wall time: 6.46 s


#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [52]:
%%time
query = f"I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Your booking with ID effe661b for 2 guests on May 6th, 2024 at 8pm has been deleted successfully.
CPU times: user 17.9 ms, sys: 0 ns, total: 17.9 ms
Wall time: 5.16 s


And we confirm that the booking has also been deleted from the dynamoDB table

In [53]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,6dca9f6d,Anna
1,4,2024-05-05,21:00,bd2554b2,Maria


#### Invoking agent using promptSessionAttributes to handle temporal information

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [54]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

'Sep-03-2024'

In [55]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John",
        "today": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking for 2 people at 8pm on September 4th, 2024 has been confirmed. Your booking ID is 80b063d4.
CPU times: user 26.9 ms, sys: 0 ns, total: 26.9 ms
Wall time: 5.71 s


And now to confirm that everything got added properly, let's retrieve the details of the last booking 

In [56]:
%%time
query = "Could you get the details for the last booking created?"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Here are the details for your booking with ID 80b063d4:

Name: John
Number of Guests: 2  
Date: September 4th, 2024
Time: 8:00 PM
CPU times: user 19.7 ms, sys: 2.65 ms, total: 22.3 ms
Wall time: 9.96 s


#### Asking the agent for food recomendations

Another good use case for our agent, is to use its reasoning capabilities to ask for some food recommendation. Let's look at a couple of examples for it

In [57]:
%%time
session_id:str = str(uuid.uuid1())
query = "What do you have for kids that don't like fries?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Based on the children's menu, some kid-friendly items that are not fries include:

- Chicken nuggets (served with ketchup or ranch dressing)
- Macaroni and cheese  
- Mini cheese quesadillas (served with mild salsa)
- Peanut butter and banana sandwich
- Veggie pita pockets (filled with hummus, cucumber, and cherry tomatoes)
- Mini cheeseburgers
- Fish sticks (served with tartar sauce)
- Grilled cheese sandwich
- Spaghetti with marinara sauce

CPU times: user 17.4 ms, sys: 3.5 ms, total: 20.9 ms
Wall time: 10.1 s


In [58]:
%%time
session_id:str = str(uuid.uuid1())
query = "I am allergic to shrimps. What can I eat at this restaurant?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


Based on the menu information, here are some dishes that do not contain shrimp that you can order at this restaurant:

- Buffalo Chicken Wings
- Caprese Salad Stuffed Avocado 
- Loaded Potato Skins
- Vegetarian Chili
- Grilled BBQ Chicken
- Southern Fried Catfish
- BBQ Pulled Pork Sandwiches





Other menu options without shrimp include:

- Chicken Nuggets
- Macaroni and Cheese
- Mini Cheese Quesadillas
- Peanut Butter and Banana Sandwich
- Veggie Pita Pockets
- Mini Cheeseburgers
- Fish Sticks
- Grilled Cheese Sandwich
- Spaghetti with Marinara Sauce
- Mini Pita Pizza



CPU times: user 21.4 ms, sys: 694 μs, total: 22.1 ms
Wall time: 14.2 s


### Invoking agent with trace enabled

We can also invoke our agent with the trace enabled to produce the details of each step being orchestrated by the Agent. Let's see all the steps executed when checking for documents in the knowledge base

In [59]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=True)
print(response)

[2024-09-03 16:45:28,430] p1345 {agent.py:170} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 03 Sep 2024 16:45:28 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ececbd4a-6a13-11ef-8971-563ddc867807',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'a717bc2d-a3db-4976-a018-c7dfe718f3a6'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a717bc2d-a3db-4976-a018-c7dfe718f3a6',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f3fb07106d0>,
 'contentType': 'application/json',
 'sessionId': 'ececbd4a-6a13-11ef-8971-563ddc867807'}


[2024-09-03 16:45:28,650] p1345 {agent.py:184} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "BHWWBLN3YG",
  "agentVersion": "DRAFT",
  "sessionId": "ececbd4a-6a13-11ef-8971-563ddc867807",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\"        You are a restaurant agent, helping clients retrieve information from their booking,create a new booking or delete an existing booking        You have been provided with a set of functions to answer the user's question.        You must call the functions in the format below:        <function_calls>        <invoke>            <tool_name>$TOOL_NAME</tool_name>            <parameters>            <$PARAMETER_NAME>$PARAMET

Based on the provided menu information, the desserts offered on the adult dinner menu at The Regrettable Experience restaurant are:

1. Classic New York Cheesecake - Creamy cheesecake with a graham cracker crust, topped with a choice of fruit compote or chocolate ganache. Allergens: Dairy, Gluten.

2. Apple Pie A la Mode - Warm apple pie with a flaky crust, served with a scoop of vanilla ice cream and a drizzle of caramel sauce. Allergens: Dairy, Gluten.

3. Chocolate Lava Cake - Rich and gooey chocolate cake with a molten center, dusted with powdered sugar and served with a scoop of raspberry sorbet. Allergens: Dairy, possible Gluten and Soy.

4. Pecan Pie Bars - Buttery shortbread crust topped with a gooey pecan filling, cut into bars for easy serving. Allergens: Dairy, Nuts, Gluten.

5. Banana Pudding Parfait - Layers of vanilla pudding, sliced bananas, and vanilla wafers, topped with whipped cream and a sprinkle of crushed nuts. Allergens: Dairy, Gluten.
CPU times: user 34.7 ms, sy

### Invoking agent with different prompt languages

The last feature that we will highlight in this notebook is the ability of LLM models to handle inputs in different languages. We've only implemented our agent in English, but wouldn't it be great if it could also handle other languages?

Good news is that it can! This is due to the LLM multi-language capabilities. And the best part is that we don't need to change anything in the agent and it will even going to respond in the requested language.

Let's try a couple of queries!

In [60]:
%%time
# Invoking agents in spanish
session_id:str = str(uuid.uuid1())
query = "อยากจะจอง book โต๊ะสำหรับ 2 ที่ วันที่ 25/07/2024 เวลา 19:30 น่ะครับ"
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Gabriela"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

ขอบคุณสำหรับการจองโต๊ะของคุณ การจองของคุณสำหรับ 2 ที่นั่งในวันที่ 25 กรกฎาคม 2024 เวลา 19:30 น. ได้รับการยืนยันแล้ว รหัสการจองของคุณคือ 471cca8f กรุณาแสดงรหัสนี้เมื่อมาถึงร้านอาหาร
CPU times: user 17.5 ms, sys: 4.21 ms, total: 21.7 ms
Wall time: 10.5 s


In [62]:
%%time
# Invoking agents in german
session_id:str = str(uuid.uuid1())
query = "Könnten Sie heute Abend einen Tisch für trei reservieren? Um 19:30 Uhr"
session_state = {
    "promptSessionAttributes": {
        "Name": "Julian",
        "Heute": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Vielen Dank für Ihre Reservierungsanfrage. Ich habe für Sie einen Tisch für 3 Personen heute Abend um 19:30 Uhr unter dem Namen Julian mit der Buchungsnummer f09a83d4 reserviert.
CPU times: user 13.8 ms, sys: 5.72 ms, total: 19.6 ms
Wall time: 14.3 s


Last, let's check our dynamoDB to see all the bookings available

In [63]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-09-04,20:00,80b063d4,John
1,2,2024-05-05,20:00,6dca9f6d,Anna
2,2,Sep-03-2024,19:30,a18dba9b,Julian
3,4,2024-05-05,21:00,bd2554b2,Maria
4,2,2024-07-25,19:30,471cca8f,Gabriela
5,3,Sep-03-2024,19:30,f09a83d4,Julian


### Next Steps

Next we will delete all the resources created